In [1]:
import sys
# sys.path.append('/Users/bernardoloureiro/template-lib')

from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import logging
import os
from dateutil import parser
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
!pip install scikit-learn
!pip install shapely
!pip install shap-hypetune
!pip install xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix,precision_score
from sklearn.preprocessing import StandardScaler
# import shap
from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
from sklearn.metrics import roc_auc_score
from scipy import stats
from xgboost import XGBClassifier
# from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.preprocessing import StandardScaler

In [15]:
#Getting the context
contextId = 'Airline_performance'
context = Helpers.getOrCreateContext(contextId=contextId, localVars=locals())

inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputDataset3",
    metadata=Metadata(input_name='Clustered data', is_required=True, tooltip='output transformation'),
    local_context=locals()
)






Departures_On_Time_Parameter = Helpers.get_or_create_input_var(
    name="Departures_On_Time",
    metadata=Metadata(input_name="Departures_On_Time", is_required=True, tooltip="All Departures_On_Time", multiple=False,datatypes=['STRING'] ,options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)




Total_scheduled_Parameter = Helpers.get_or_create_input_var(
    name="Total_scheduled",
    metadata=Metadata(input_name="Total_scheduled", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)



Total_departed_Parameter = Helpers.get_or_create_input_var(
    name="Total_departed",
    metadata=Metadata(input_name="Total_departed", is_required=True, tooltip="All Departures_On_Time", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)





Cancellations_Parameter = Helpers.get_or_create_input_var(
    name="Cancellations",
    metadata=Metadata(input_name="Cancellations", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Arrivals_Delayed_Parameter = Helpers.get_or_create_input_var(
    name="Arrivals_Delayed",
    metadata=Metadata(input_name="Arrivals_Delayed", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)





Cancelled_perc_Parameter = Helpers.get_or_create_input_var(
    name="Cancelled_perc",
    metadata=Metadata(input_name="Cancelled_perc", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)


departure_delay_perc_Parameter = Helpers.get_or_create_input_var(
    name="departure_delay_perc",
    metadata=Metadata(input_name="departure_delay_perc", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
)


arrival_delay_perc_Parameter = Helpers.get_or_create_input_var(
    name="arrival_delay_perc",
    metadata=Metadata(input_name="arrival_delay_perc", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)


Dep_Arr_delay_Parameter = Helpers.get_or_create_input_var(
    name="Dep_Arr_delay",
    metadata=Metadata(input_name="Dep_Arr_delay", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)



Btwness_Parameter = Helpers.get_or_create_input_var(
    name="Btwness",
    metadata=Metadata(input_name="Btwness", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)


closeness_Parameter = Helpers.get_or_create_input_var(
    name="closeness",
    metadata=Metadata(input_name="closeness", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 


Deg_centrality_Parameter = Helpers.get_or_create_input_var(
    name="Deg_centrality",
    metadata=Metadata(input_name="Deg_centrality", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 









label_Parameter = Helpers.get_or_create_input_var(
    name="label",
    metadata=Metadata(input_name="label", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)



outputDatasetParameter=Helpers.get_or_create_output_dataset(
 name="outputDataset3",
    metadata=Metadata(input_name='Xtrain', is_required=True, tooltip='Dataset name to be created after the transformation'),
    local_context=locals()
)

# outputDatasetParameter2=Helpers.get_or_create_output_dataset(
#  name="outputDataset4",
#     metadata=Metadata(input_name='XTest', is_required=True, tooltip='Dataset name to be created after the transformation'),
#     local_context=locals()
# )

# outputDatasetParameter3=Helpers.get_or_create_output_dataset(
#  name="outputDataset5",
#     metadata=Metadata(input_name='YTrain', is_required=True, tooltip='Dataset name to be created after the transformation'),
#     local_context=locals()
# )

# outputDatasetParameter4=Helpers.get_or_create_output_dataset(
#  name="outputDataset6",
#     metadata=Metadata(input_name='YTest', is_required=True, tooltip='Dataset name to be created after the transformation'),
#     local_context=locals()
# )



#Model Save

model_to_saveParameter = Helpers.get_or_create_input_var(
    name="model_to_save",
    metadata=Metadata(input_name="XGB Model", is_required=False, tooltip="If you want your model to be saved and be used on for prediction, it is required.", multiple=False, datatypes=['STRING'], options=['CONSTANT']),
    local_context=locals()
)

In [16]:
#Getting the parameters
airline_clust=inputDatasetParameter.value



Departures_On_Time =Departures_On_Time_Parameter.value

Cancellations=    Cancellations_Parameter.value

Arrivals_Delayed = Arrivals_Delayed_Parameter.value

Total_scheduled=Total_scheduled_Parameter.value
Total_departed= Total_departed_Parameter.value
Cancelled_perc = Cancelled_perc_Parameter.value
departure_delay_perc= departure_delay_perc_Parameter.value
arrival_delay_perc = arrival_delay_perc_Parameter.value
Dep_Arr_delay = Dep_Arr_delay_Parameter.value
Btwness = Btwness_Parameter.value
closeness = closeness_Parameter.value
Deg_centrality = Deg_centrality_Parameter.value








label = label_Parameter.value

outputDataset3=outputDatasetParameter.value
# outputDataset4=outputDatasetParameter.value
# outputDataset5=outputDatasetParameter.value
# outputDataset6=outputDatasetParameter.value

model_to_save=model_to_saveParameter.value


In [17]:
model_to_save

In [5]:
airline_clust =  Helpers.getEntityData(context, airline_clust)

In [6]:
Xtrain = airline_clust.drop(['label'],axis = 1)

Ytrain=airline_clust['label']

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(Xtrain,Ytrain,test_size=0.2,random_state=1200,stratify = Ytrain)
print(X_train.shape , X_test.shape)

(1424, 12) (356, 12)


In [8]:
sc = StandardScaler()
X_train_norm = sc.fit_transform(X_train)
X_test_norm = sc.transform(X_test)

In [9]:
clf_xgb = XGBClassifier(n_estimators=150, random_state=0, verbosity=0, n_jobs=-1)

In [10]:
param_dist = {
    'learning_rate': stats.uniform(0.01, 0.1),
    'num_leaves': stats.randint(35,40),
    'max_depth': [10, 18]
}


In [12]:
model = BoostRFE(
    clf_xgb, param_grid=param_dist, min_features_to_select=1, step=1,
    n_iter=20, sampling_seed=0,greater_is_better=False,
    importance_type='shap_importances', train_importance=False
)
model.fit(X_train_norm, Y_train, eval_set=[(X_test_norm, Y_test)], verbose=0)


3 trials detected for ('learning_rate', 'num_leaves', 'max_depth')

trial: 0001 ### iterations: 00150 ### eval_score: 0.02878
trial: 0002 ### iterations: 00150 ### eval_score: 0.02646
trial: 0003 ### iterations: 00150 ### eval_score: 0.06749


BoostRFE(estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
                                 colsample_bylevel=None, colsample_bynode=None,
                                 colsample_bytree=None,
                                 early_stopping_rounds=None,
                                 enable_categorical=False, eval_metric=None,
                                 feature_types=None, gamma=None, gpu_id=None,
                                 grow_policy=None, importance_type=None,
                                 interaction_constraints=None,
                                 learning_rate=None, max_bin=N...
                                 num_parallel_tree=None, predictor=None,
                                 random_state=0, ...),
         importance_type='shap_importances', min_features_to_select=1, n_iter=3,
         param_grid={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f596c9aa3e0>,
                     'max_depth': [10, 18],
                     'num_leaves': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f596c9a9930>},
         sampling_seed=0, train_importance=False)

In [13]:
result = pd.DataFrame(classification_report(Y_test, model.predict(X_test_norm),output_dict=True)).transpose()

In [ ]:
if model_to_save is None or len(model_to_save) == 0 :
    { print('No model to be saved')}
else:
    import pickle
    artifactsDir = Helpers.getOrCreateArtifactsDir(context, artifactsId = model_to_save)
    with open(artifactsDir + '/XGB_Shap_.pkl', 'wb') as handle:
        pickle.dump(model, handle)

filename = 'XGB_model_shap.sav'
pickle.dump(model, open(filename, 'wb'))

result= pd.DataFrame(model.predict(X_test_norm))

In [ ]:
outputCollection = Helpers.createOutputCollection(context)
out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset3, dataFrame=result)

# out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset3, dataFrame=Xtrain)
# out2 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset4, dataFrame=X_test_norm)
# out3 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset5, dataFrame=Y_train)
# out4 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset6, dataFrame=Y_test)
outputCollection.addTemplateOutput(out)
# outputCollection.addTemplateOutput(out2)
# outputCollection.addTemplateOutput(out3)
# outputCollection.addTemplateOutput(out4)
Helpers.save(context)